
The feature selection notebook does a filter followed by a wrapper. It's capable of doing the filter on more than one file. The variable files are called vars1.csv, vars2.csv ...

The filter runs separately on each vars file and keeps the top num_filter variables from each file. If there are more than one vars files we'll again select the top num_filter variables across all the vars.csv files.

If balance = 0 the entire files are used. If balance != 0 then balance is the ratio of goods to bads retained for the rest of the feature selection. We keep all the rare class (bads) and downsample the goods. 

BUT RIGHT NOW I'VE GOT A BUG SOMEWHERE: balance !=0 doesn't work when num_files > 1. I've got an index problem.

So you can run with multiple variables files only when balance=0, or you can run with only one variable file and set balance how you like.

In [1]:
# load libraries
# %pip install mlxtend

import pandas as pd
import numpy as np
import scipy.stats as sps
import matplotlib.pyplot as plt
import datetime as dt
import gc
from sklearn.ensemble import RandomForestClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
%matplotlib inline
start_time = dt.datetime.now()

In [2]:
# set some parameters
num_files = 1
num_filter = 100
num_wrapper = 30
# keep balance = 0 when num_files > 1 until I figure out this bug. Maybe somebody else can figure it out.
balance = 0
detect_rate = .03
index_name = 'record'
y_name = 'fraud_label'
good_label = 0
bad_label = 1

In [3]:
%cd D:\USC\4th semester\DSO562 fraud

D:\USC\4th semester\DSO562 fraud


In [4]:
file_name = 'vars.csv'
df = pd.read_csv(file_name)

## Run a filter on all the files

In [5]:
%%time
for i in range(num_files):
    print('work on file ', file_name)
    df = df[df[index_name] <= 833508] # remove the last two months as the out-of-time data (OOT)
    df = df[df[index_name] >= 38512] # remove the first 2 weeks of records since their variables aren't well formed
    df['RANDOM'] = np.random.ranf(len(df)) # add a random number variable to make sure it doesn't come up as important
    goods = df[df[y_name] == good_label]
    bads = df[df[y_name] == bad_label]
    del df # don't need this file anymore
    num_goods = len(goods)
    num_bads = len(bads)
    num_vars = len(bads.columns)-2
    if(balance != 0):
        if(i == 0):
            num_goods_desired = int(min(num_goods,num_bads*balance))
            goods = goods.sample(n=num_goods_desired,random_state=1)
            goods_keep = list(goods.index)
            goods_keep.sort()
    
        if(i > 0):
            goods.set_index(index_name, inplace=True)
            goods.sort_index(inplace=True)
            goods = goods.iloc[goods_keep] # THERE'S SOMETHING WRONG HERE AND I CAN'T FIGURE IT OUT
            
    df_sampled = pd.concat([goods,bads])
    df_sampled.set_index(index_name, inplace=True)
    df_sampled.sort_index(inplace=True)
    if(i == 0): # if first time through need to initialize some stuff
        Y = pd.DataFrame(df_sampled[y_name], index=df_sampled.index)
        filter_score = pd.DataFrame(np.zeros((num_vars+1,2)))
        filter_score.columns = ['variable','score']   
        j = 0
        for column in df_sampled:
            filter_score.loc[j,'variable'] = column
            filter_score.loc[j,'score'] = sps.ks_2samp(goods[column],bads[column])[0]
            j = j+1
            
        filter_score.sort_values(by=['score'], ascending=False, inplace=True)
        vars_keep = list(filter_score['variable'][1:num_filter+1])
        print(filter_score.head(num_filter))
        df_top = df_sampled.filter(vars_keep, axis=1)
            
    if(i > 0): # if more than one variable file we use this loop
        new = pd.DataFrame(np.zeros((num_vars+1,2)))
        new.columns = ['variable','score']   
        j = 0
        for column in df_sampled:
            new.loc[j,'variable'] = column
            new.loc[j,'score'] = sps.ks_2samp(goods[column],bads[column])[0]
            j = j+1
        
        new.sort_values(by=['score'], ascending=False, inplace=True)
        vars_keep = list(new['variable'][1:num_filter+1])
        print(new.head(num_filter))
        data_new_top = df_sampled.filter(vars_keep, axis=1)
        df_top = pd.concat([df_top,data_new_top], axis=1)
        filter_score = pd.concat([filter_score,new])
    
    del goods # delete these before starting the next file, if any
    del bads
    gc.collect()

work on file  vars.csv
                               variable     score
9                           fraud_label  1.000000
37                    address_day_since  0.334096
65                fulladdress_day_since  0.333210
43                     address_count_30  0.332725
71                 fulladdress_count_30  0.332032
..                                  ...       ...
124               ssn_firstname_count_3  0.172088
152                     ssn_dob_count_3  0.172059
166                    ssn_name_count_3  0.171928
131                ssn_lastname_count_3  0.171914
265  fulladdress_homephone_count_0_by_7  0.171823

[100 rows x 2 columns]
Wall time: 2min 24s


In [6]:
filter_score.sort_values(by=['score'], ascending=False, inplace=True)

In [7]:
filter_score.head(30)

,variable,score
9,fraud_label,1.000000
37,address_day_since,0.334096
65,fulladdress_day_since,0.333210
43,address_count_30,0.332725
71,fulladdress_count_30,0.332032
42,address_count_14,0.322252
70,fulladdress_count_14,0.321756
41,address_count_7,0.301445
69,fulladdress_count_7,0.301368
195,address_count_0_by_30,0.291997


In [8]:
filter_score.tail(10)

,variable,score
116,homephone_name_dob_count_1,0.005650
108,dob_homephone_count_0,0.001986
136,ssn_address_count_0,0.001858
143,ssn_zip5_count_0,0.001857
80,name_fulladdress_count_0,0.001850
115,homephone_name_dob_count_0,0.001821
94,fulladdress_dob_count_0,0.001812
157,ssn_homephone_count_0,0.001793
87,name_homephone_count_0,0.001782
171,ssn_fulladdress_count_0,0.001774


In [9]:
filter_score.shape

(361, 2)

In [10]:
filter_score.sort_values(by=['score'], ascending=False, inplace=True)
vars_keep = list(filter_score['variable'][num_files:num_filter+3])
print(i,' vars_keep:',vars_keep)

0  vars_keep: ['address_day_since', 'fulladdress_day_since', 'address_count_30', 'fulladdress_count_30', 'address_count_14', 'fulladdress_count_14', 'address_count_7', 'fulladdress_count_7', 'address_count_0_by_30', 'fulladdress_count_0_by_30', 'address_count_0_by_14', 'fulladdress_count_0_by_14', 'fulladdress_count_3', 'address_count_3', 'address_count_0_by_7', 'fulladdress_count_0_by_7', 'address_count_1', 'fulladdress_count_1', 'fulladdress_count_0_by_3', 'address_count_0_by_3', 'fulladdress_homephone_day_since', 'fulladdress_homephone_count_30', 'ssn_dob_day_since', 'ssn_dob_count_30', 'name_dob_day_since', 'name_dob_count_30', 'ssn_day_since', 'ssn_count_30', 'ssn_name_dob_day_since', 'ssn_firstname_day_since', 'ssn_lastname_day_since', 'ssn_name_dob_count_30', 'ssn_firstname_count_30', 'ssn_lastname_count_30', 'ssn_name_day_since', 'ssn_name_count_30', 'fulladdress_homephone_count_14', 'address_count_1_by_30', 'fulladdress_count_1_by_30', 'name_dob_count_14', 'ssn_dob_count_14', 

In [11]:
df_keep = df_top.filter(vars_keep, axis=1)
df_keep.head()

,address_day_since,fulladdress_day_since,address_count_30,fulladdress_count_30,address_count_14,fulladdress_count_14,address_count_7,fulladdress_count_7,address_count_0_by_30,fulladdress_count_0_by_30,...,dob_count_14,name_dob_count_3,name_dob_count_0_by_7,ssn_count_3,ssn_firstname_count_3,ssn_dob_count_3,ssn_name_count_3,ssn_lastname_count_3,fulladdress_homephone_count_0_by_7,ssn_name_dob_count_3
record,,,,,,,,,,,,,,,,,,,,,
38512,365.0,365.0,1,1,1,1,1,1,30.0,30.0,...,1,1,7.0,1,1,1,1,1,7.0,1
38513,365.0,365.0,1,1,1,1,1,1,30.0,30.0,...,2,1,7.0,1,1,1,1,1,7.0,1
38514,365.0,365.0,1,1,1,1,1,1,30.0,30.0,...,2,1,7.0,1,1,1,1,1,7.0,1
38515,365.0,365.0,1,1,1,1,1,1,30.0,30.0,...,1,1,7.0,1,1,1,1,1,7.0,1
38516,365.0,365.0,1,1,1,1,1,1,30.0,30.0,...,1,1,7.0,1,1,1,1,1,7.0,1


In [12]:
df_keep.shape

(794997, 100)

In [13]:
Y.head()

,fraud_label
record,
38512,0
38513,0
38514,0
38515,0
38516,0


In [14]:
Y = Y.values
X = df_keep

In [15]:
print('time to here:', dt.datetime.now() - start_time)

time to here: 0:18:12.888695


In [16]:
print(X.shape,Y.shape)

(794997, 100) (794997, 1)


In [18]:
type(Y)

numpy.ndarray

In [19]:
X.to_csv('X.csv',index=False)

In [21]:
np.save('Y.npy', Y)

In [23]:
df_keep.shape

(794997, 100)

In [25]:
type(X)

pandas.core.frame.DataFrame

# Below was not run in Jupyter notebook due to memory constraint

# See part two in Google Colab

In [15]:
# I'd like to define a scoring for the wrapper that's KS, but I haven't gotten around to this yet.
# def KSscore(classifier, x,y)
#     goods = 

In [17]:
def fdr(classifier, x, y, cutoff=detect_rate):
# Calculates FDR score for the given classifier on dataset x and y with cutoff value
# get the probability list from the given classifier
    return fdr_prob(y, classifier.predict_proba(x), cutoff)
def fdr_prob(y, y_prob, cutoff=detect_rate):
    if len(y_prob.shape) != 1:    # sometimes the proba list can contain many columns, one for each category
        y_prob = y_prob[:, -1:]   # only the last one (fraud_label==1) is used here.
    num_fraud = len(y[y == 1])    # count the total nunber of frauds   
# sort the proba list from high to low while retain the true (not predicted) fraud label
    sorted_prob = np.asarray(sorted(zip(y_prob, y), key=lambda x: x[0], reverse=True))
    cutoff_bin = sorted_prob[0:int(len(y) * cutoff), 1:]  # 3% cutoff
# return the FDR score (#fraud_in_cutoff / #total_fraud)
    return len(cutoff_bin[cutoff_bin == 1]) / num_fraud   

In [21]:
%%time
# If you're doing forward selection it's enough to stop at num_wrapper variables. 
# If you're doing backward selection you need to go through all the variables to get 
# a sorted list of num_wrapper variables.
nfeatures = len(X.columns)
clf = RandomForestClassifier(n_estimators=5) # simple, fast nonlinear model for the wrapper
# sfs = SFS(clf,k_features=num_wrapper,forward=True,verbose=2,scoring=fdr,cv=2,n_jobs=-1) # forward selection
sfs = SFS(clf,k_features=1,forward=False,verbose=2,scoring=fdr,cv=2) # backward selection
sfs.fit(X,Y)

D:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
D:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversion

D:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change

D:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change

D:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change

D:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change

...

D:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change

D:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change

Wall time: 11h 2min 59s



STOPPING EARLY DUE TO KEYBOARD INTERRUPT...

SequentialFeatureSelector(cv=2,
                          estimator=RandomForestClassifier(n_estimators=5),
                          forward=False,
                          scoring=<function fdr at 0x000001B862770950>,
                          verbose=2)

In [19]:
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
fig1 = plot_sfs(sfs.get_metric_dict(),kind='std_dev', figsize=(15, 6))
plt.xticks(np.arange(0, len(X.columns), step=5))
plt.ylim([0., .7])
plt.xlim(0,num_wrapper)
plt.title('Stepwise Selection')
plt.grid()
plt.savefig('FS_fdr.png')
plt.show()

AttributeError: SequentialFeatureSelector has not been fitted, yet.

In [19]:
vars_FS = pd.DataFrame.from_dict(sfs.get_metric_dict()).T
vars_FS

,feature_idx,cv_scores,avg_score,feature_names,ci_bound,std_dev,std_err
1,"(6,)","[0.08984851123106391, 0.08984851123106391]",0.0898485,"(address_count_7,)",0,0,0
2,"(6, 7)","[0.08984851123106391, 0.08984851123106391]",0.0898485,"(address_count_7, fulladdress_count_7)",0,0,0
3,"(6, 7, 44)","[0.08984851123106391, 0.08984851123106391]",0.0898485,"(address_count_7, fulladdress_count_7, fulladd...",0,0,0
4,"(6, 7, 44, 45)","[0.08984851123106391, 0.08984851123106391]",0.0898485,"(address_count_7, fulladdress_count_7, fulladd...",0,0,0
5,"(6, 7, 44, 45, 46)","[0.08984851123106391, 0.08984851123106391]",0.0898485,"(address_count_7, fulladdress_count_7, fulladd...",0,0,0
6,"(6, 7, 44, 45, 46, 47)","[0.08984851123106391, 0.08984851123106391]",0.0898485,"(address_count_7, fulladdress_count_7, fulladd...",0,0,0
7,"(6, 7, 44, 45, 46, 47, 48)","[0.08984851123106391, 0.08984851123106391]",0.0898485,"(address_count_7, fulladdress_count_7, fulladd...",0,0,0
8,"(6, 7, 44, 45, 46, 47, 48, 49)","[0.08984851123106391, 0.08984851123106391]",0.0898485,"(address_count_7, fulladdress_count_7, fulladd...",0,0,0
9,"(6, 7, 44, 45, 46, 47, 48, 49, 51)","[0.08984851123106391, 0.08984851123106391]",0.0898485,"(address_count_7, fulladdress_count_7, fulladd...",0,0,0
10,"(6, 7, 44, 45, 46, 47, 48, 49, 51, 101)","[0.08984851123106391, 0.08984851123106391]",0.0898485,"(address_count_7, fulladdress_count_7, fulladd...",0,0,0


In [42]:
vars_final = list(vars_FS.loc[num_wrapper, 'feature_names'])
vars_final

['address_count_30',
 'fulladdress_count_30',
 'address_count_14',
 'fulladdress_count_14',
 'address_count_7',
 'fulladdress_count_7',
 'fulladdress_count_0_by_30',
 'fulladdress_unique_count_for_dob_homephone_60',
 'fulladdress_unique_count_for_ssn_lastname_60',
 'fulladdress_unique_count_for_ssn_fulladdress_60',
 'fulladdress_unique_count_for_ssn_name_60',
 'fulladdress_unique_count_for_ssn_zip5_60',
 'fulladdress_unique_count_for_ssn_60',
 'fulladdress_unique_count_for_ssn_name_dob_60',
 'fulladdress_unique_count_for_name_fulladdress_60',
 'fulladdress_unique_count_for_dob_homephone_30',
 'fulladdress_unique_count_for_name_dob_60',
 'fulladdress_unique_count_for_fulladdress_dob_60',
 'fulladdress_unique_count_for_ssn_name_30',
 'fulladdress_unique_count_for_ssn_zip5_30',
 'address_count_0_by_14',
 'fulladdress_unique_count_for_ssn_name_dob_30',
 'fulladdress_unique_count_for_name_dob_30',
 'fulladdress_unique_count_for_fulladdress_dob_30',
 'fulladdress_unique_count_for_dob_homepho

In [43]:
vars_FS.to_csv('VarsFS.csv', index=False)

In [44]:
sfs.subsets_

{1: {'feature_idx': (6,),
  'cv_scores': array([0.11997214, 0.11979801]),
  'avg_score': 0.11988507748563468,
  'feature_names': ('address_count_7',)},
 2: {'feature_idx': (6, 7),
  'cv_scores': array([0.11997214, 0.11979801]),
  'avg_score': 0.11988507748563468,
  'feature_names': ('address_count_7', 'fulladdress_count_7')},
 3: {'feature_idx': (6, 7, 44),
  'cv_scores': array([0.11997214, 0.11979801]),
  'avg_score': 0.11988507748563468,
  'feature_names': ('address_count_7',
   'fulladdress_count_7',
   'fulladdress_unique_count_for_dob_homephone_7')},
 4: {'feature_idx': (6, 7, 44, 45),
  'cv_scores': array([0.11997214, 0.11979801]),
  'avg_score': 0.11988507748563468,
  'feature_names': ('address_count_7',
   'fulladdress_count_7',
   'fulladdress_unique_count_for_dob_homephone_7',
   'fulladdress_unique_count_for_ssn_fulladdress_7')},
 5: {'feature_idx': (6, 7, 44, 45, 46),
  'cv_scores': array([0.11997214, 0.11979801]),
  'avg_score': 0.11988507748563468,
  'feature_names': ('ad

In [45]:
vars_final.append('index_name')
vars_final.append('y_name')
vars_final

['address_count_30',
 'fulladdress_count_30',
 'address_count_14',
 'fulladdress_count_14',
 'address_count_7',
 'fulladdress_count_7',
 'fulladdress_count_0_by_30',
 'fulladdress_unique_count_for_dob_homephone_60',
 'fulladdress_unique_count_for_ssn_lastname_60',
 'fulladdress_unique_count_for_ssn_fulladdress_60',
 'fulladdress_unique_count_for_ssn_name_60',
 'fulladdress_unique_count_for_ssn_zip5_60',
 'fulladdress_unique_count_for_ssn_60',
 'fulladdress_unique_count_for_ssn_name_dob_60',
 'fulladdress_unique_count_for_name_fulladdress_60',
 'fulladdress_unique_count_for_dob_homephone_30',
 'fulladdress_unique_count_for_name_dob_60',
 'fulladdress_unique_count_for_fulladdress_dob_60',
 'fulladdress_unique_count_for_ssn_name_30',
 'fulladdress_unique_count_for_ssn_zip5_30',
 'address_count_0_by_14',
 'fulladdress_unique_count_for_ssn_name_dob_30',
 'fulladdress_unique_count_for_name_dob_30',
 'fulladdress_unique_count_for_fulladdress_dob_30',
 'fulladdress_unique_count_for_dob_homepho

In [46]:
X_final = df_keep.filter(vars_final, axis=1)
X_final.shape

(45944, 30)

In [47]:
X_final.to_csv('vars_final.csv', index=False)

In [48]:
print('time to here:', dt.datetime.now() - start_time)

time to here: 6:51:44.499455
